# Dynamic Programming 

날짜 : 23/03/24



In [1]:
# install gitPython
import os, sys, time
!pip install gitPython
# clone my repository
import git
!git clone https://github.com/sungbinlim/RLclass.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 5.6 MB/s eta 0:00:00
Cloning into 'RLclass'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 40 (delta 8), reused 32 (delta 0), pack-reused 0
Unpacking objects: 100% (40/40), 7.45 KiB | 476.00 KiB/s, done.


`dynamics.py`, `grid_world.py` 있음

In [3]:
%cd /content/RLclass/STAT436/grid_world/ # 해당 폴더에서 작업하겠다.

[Errno 2] No such file or directory: '/content/RLclass/STAT436/grid_world/ # 해당 폴더에서 작업하겠다.'
/content/RLclass/STAT436/grid_world


In [5]:
# append package address
sys.path.append("/content/RLclass/STAT436/grid_world/") # wd에 있는 패키지 로드
from grid_world import *
from dynamics import *

## Index

1. Bellman Equation (about v)

2. Bellman Equation solving (Hacking Dynamics)

3. Optimal Policy Exploration

## Hacking Dynamics

Bellman equation:

$$
\mathbf{v}=\mathbf{P}_{\text{reward}}\mathbf{r}+\gamma\mathbf{P}_{\text{value}}\mathbf{v}
$$

where $\mathbf{v}\in\mathbb{R}^{|\mathcal{S}|}$, $\mathbf{r}\in\mathbb{R}^{|\mathcal{R}|}$, $\mathbf{P}_{\text{reward}}\in\mathbb{R}^{|\mathcal{S}|\times|\mathcal{R}|}$, and $\mathbf{P}_{\text{value}}\in\mathbb{R}^{|\mathcal{S}|\times|\mathcal{S}|}$ such that 

\begin{aligned}
\left(\mathbf{P}_{\text{reward}}\right)_{jq}=\sum_{a_{p}\in\mathcal{A}}\pi(a_{p}|s_{j})\sum_{s_{i}\in\mathcal{S}}\mathcal{P}(s_{i},r_{q}|s_{j},a_{p})=\boldsymbol{\pi}_{p}\mathbf{P}_{i,j,p,q}\mathbf{1}_{i}\\
\left(\mathbf{P}_{\text{value}}\right)_{ji}=\sum_{a_{p}\in\mathcal{A}}\pi(a_{p}|s_{j})\sum_{r_{q}}\mathcal{P}(s_{i},r_{q}|s_{j},a_{p})=\boldsymbol{\pi}_{p}\mathbf{P}_{i,j,p,q}\mathbf{1}_{q}
\end{aligned}

If $\gamma \in (0, 1)$ then we can solve the equation as follows
$$
\mathbf{v}=(I-\gamma\mathbf{P}_{\text{value}})^{-1}\mathbf{P}_{\text{reward}}\mathbf{r}
$$

To compute action-value function $q_{\pi}(s,a)$, we use the following formula:
$$
q_{\pi}(s,a)=\sum_{s'\in\mathcal{S},r\in\mathcal{R}} [r +\gamma v_{\pi}(s')]\mathcal{P}(s',r|s,a)
$$

합(policy), 곱, 합(dynamics) 으로 이루어짐

S(j) : 12차원

A(p): 4차원

R(q) : 1,-1, 0 (for cumulative reward), 3차원

(P_reward)_jq : j*q 행렬이므로 j가 앞에나온는게 계산에 편리함.

**그러므로,**

policy는 12 * 4 차원


- Reward matrix

4차원 짜리 tensor를 sum(marginalize)해주면 3차원이 된다. (for Si : Si 는 next timestep state)

3차원짜리 tensor를 sum(marginalize)해주면 2차원이 된다. (for ap : : ap 는 action)

j라는 state에서 q라는 reward를 얻을 확률

그러므로 P_reward * r 은 각각의 r에 대한 probability를 곱해준, reward에 대한 expectation이 된다.

12(s_j).3.12(s_i).4 --> s_i --> 12.3.4 -->  a --> 12.3

  

   



- Value matrix 

j라는 state에서 i라는 state의 value값을 얻을 확률

미래 발생할수 있는 모든 value의 기댓값 (i번째로 끌어옴. 할인율을 통해)

12.3.12.4 (dynamics) --> r -> 12.12.4 --> pi --> 12.12

γ(gamma) : 할인율이 1보다 작을때 해당 행렬은 invertible

In [ ]:
import numpy as np
class pi_dynamics:
    def __init__(self, pi, gamma, reward, dynamics): # class 기본정보 
        # policy
        self.pi = pi 
        self.gamma = gamma
        self.reward = reward
        grid_world_dynamics = dynamics() # dynamics.py 에 속한 dynamics class를 불러오고
        # dynamics
        self.dynamics = grid_world_dynamics.dynamics # 4차원짜리 텐서 가져옴
        self.pi_dynamics = np.zeros_like(self.dynamics) # [current_state, next_state, action, value(reward아님?)] # dynamics.py 에서 이렇게 세팅되어있음.
        self.P_reward = np.zeros((12, 3))
        self.P_value = np.zeros((12, 12))
        self.pi_dynamics, self.P_reward, self.P_value = self.update_all() # 위에서 가져온 pi, dynamics를 가지고 P_reward랑 P_value를 계산하는 과정 

    def update_pi_dynamics(self, pi_dynamics):
        """
        compute pi * dynamics
        """  

        # if i = p , for j, a
        # 수식에서는 i가 next state인데, 코드에서는 i 가 current state이다.
        for i in range(12):
          for a in range(4):
            # 4차원 텐서를 2차원 텐서로 바꿔줌 via ':' notation
            """
            broadcasting 
            """
            pi_dynamics[i,:,a,:] = self.pi[i,a] * self.dynamics[i,:,a,:] # pi가 이렇게 설정하심. i번째 state에서 a action을 선택할 것이다. 
        """
        더 빨리 짜는 코드가 존재함. for문이 아닌 다른 iter tools 조교님께 질문.
        """
        return pi_dynamics

    def compute_P_reward(self, P_reward):
        """
        return P_reward[next_state, reward]: marginalize pi_dynamics in state
        """
        """
        marginalization
        """

        for j in range(12):
          for r in range(3):
            # next state, reward 에 대한 summation
            P_reward[j,r] = np.sum(self.pi_dynamics[j,:,:,r])
        return P_reward

    def compute_P_value(self, P_value):
        """
        return P_value[next_state, state]: marginalize pi_dynamics in reward
        """
        # state -> state

        for j in range(12):
          for i in range(12):
            P_value[j,i] = np.sum(self.pi_dynamics[j,i,:,:])
        return P_value

    def update_all(self):
        return self.update_pi_dynamics(self.pi_dynamics), self.compute_P_reward(self.P_reward), self.compute_P_value(self.P_value)

    def compute_state_value(self):
        """
        return state-value function via closed-form formula
        """
        coefficient = np.eye(12) - self.gamma * self.P_value
        inv_coeff = np.linalg.inv(coefficient)
        state_value = inv_coeff @ self.P_reward @ self.reward
        return state_value

    def compute_action_value(self):
        """
        return action-value function using state-value function
        """
        state_value = self.compute_state_value()
        expectation_reward = np.zeros((12,3))
        expectation_value = np.zeros((12,4))
        # 여기서부터 직접 짜보라고 하심.
        return action_value

## Let's run Grid World (using `randomAgent`)

- import `dynamics` from `dynamics.py`
- import `run_grid_world` from `grid_world.py`

In [ ]:
gamma = 0.99

# random policy function
pi = np.array([0.25, 0.25, 0.25, 0.25]) #up, left, right, down
pi = np.reshape(np.tile(pi, 12), (12, 4))
# reward
reward = np.array([1, 0 ,-1])

# initialize dynamics with randomAgent
init_dynamics = dynamics
init_pi_dynamics = pi_dynamics(pi, gamma, reward, init_dynamics) # pi_dynamics initial arguments 넣어줌.
state_value = init_pi_dynamics.compute_state_value()
action_value = init_pi_dynamics.compute_action_value()

In [ ]:
# run randomAgent
run_grid_world(pi, state_value, action_value)

In [ ]:
print(state_value)

In [ ]:
print(action_value)

## Then, How do we find optimal policy? (Update)